# Demo "Relaxed Releases with DB-Workloadtests": Preparation

## Disclaimer
THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED
TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR
PARTICULAR PURPOSE. 

CODE IS WRITTEN FOR DEMO PURPOSES 

(C) Martin Guth and others, 2020

## Prerequisites
* SQL Server 2008 or newer (tested with SQL Server 2019 on Linux resp Docker)
* [StackOverflow-Database](https://www.brentozar.com/archive/2015/10/how-to-download-the-stack-overflow-database-via-bittorrent/) (tested with StackOverflow2010 aka small edition)

## Steps


StackOverflow is shipped with CompatibilityLevel SQL Server 2008. Raise that to the newest version on your system (here 150=SQL2019) in order to be able to use the complete featureset.

In [1]:
ALTER DATABASE [StackOverflow2010] SET COMPATIBILITY_LEVEL = 150
GO

Commands completed successfully.

Total execution time: 00:00:00.095

Create Procedures for typical Insert-, Update- and Select-Statements
(Code adapted from [Brent Ozar](https://www.brentozar.com/archive/2015/05/how-to-fake-load-tests-with-sqlquerystress/))

Anecdote: In the beginning I wrote this code in a manner that an INSERT generates a lot of random values. This technique turned out be not suited for my demos as it resulted in a different shape of data after each replay and accordingly different results of selects. This is IMHO not a good basis for an A/B-Test...the same applies for Brent Ozar's procedure usp_RandomQ. This leads to completely different statements runnning in each replay rendering replay results not suitable for comparison.

In [15]:
USE  StackOverflow2010;
GO
/*
	Code from https://www.brentozar.com/archive/2015/05/how-to-fake-load-tests-with-sqlquerystress/
	All Credit goes to the great Brent Ozar....Thanks Brent

*/

IF OBJECT_ID('dbo.usp_Q7521') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q7521 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q7521 @UserId INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/7521/how-unsung-am-i */

-- How Unsung am I?
-- Zero and non-zero accepted count. Self-accepted answers do not count.

select
    count(a.Id) as [Accepted Answers],
    sum(case when a.Score = 0 then 0 else 1 end) as [Scored Answers],  
    sum(case when a.Score = 0 then 1 else 0 end) as [Unscored Answers],
    sum(CASE WHEN a.Score = 0 then 1 else 0 end)*1000 / count(a.Id) / 10.0 as [Percentage Unscored]
from
    Posts q
  inner join
    Posts a
  on a.Id = q.AcceptedAnswerId
where
      a.CommunityOwnedDate is null
  and a.OwnerUserId = @UserId
  and q.OwnerUserId != @UserId
  and a.PostTypeId = 2
END
GO


IF OBJECT_ID('dbo.usp_Q36660') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q36660 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q36660 @Useless INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/36660/most-down-voted-questions */

select top 20 count(v.PostId) as 'Vote count', v.PostId AS [Post Link],p.Body
from Votes v 
inner join Posts p on p.Id=v.PostId
where PostTypeId = 1 and v.VoteTypeId=3
group by v.PostId,p.Body
order by 'Vote count' desc

END
GO


IF OBJECT_ID('dbo.usp_Q949') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q949 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q949 @UserId INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/949/what-is-my-accepted-answer-percentage-rate */

SELECT 
	CASE WHEN (SELECT Count(*) FROM Posts WHERE OwnerUserId = @UserId AND PostTypeId = 2) > 0 THEN
    (CAST(Count(a.Id) AS float) / (SELECT Count(*) FROM Posts WHERE OwnerUserId = @UserId AND PostTypeId = 2) * 100)  END AS AcceptedPercentage
FROM
    Posts q
  INNER JOIN
    Posts a ON q.AcceptedAnswerId = a.Id
WHERE
    a.OwnerUserId = @UserId
  AND
    a.PostTypeId = 2

END
GO



IF OBJECT_ID('dbo.usp_Q466') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q466 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q466 @Useless INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/466/most-controversial-posts-on-the-site */
set nocount on 

declare @VoteStats table (PostId int, up int, down int) 

insert @VoteStats
select
    PostId, 
    up = sum(case when VoteTypeId = 2 then 1 else 0 end), 
    down = sum(case when VoteTypeId = 3 then 1 else 0 end)
from Votes
where VoteTypeId in (2,3)
group by PostId

set nocount off


select top 100 p.Id as [Post Link] , up, down from @VoteStats 
join Posts p on PostId = p.Id
where down > (up * 0.5) and p.CommunityOwnedDate is null and p.ClosedDate is null
order by up desc
END
GO




IF OBJECT_ID('dbo.usp_Q947') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q947 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q947 @UserId INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/947/my-comment-score-distribution */

SELECT 
    Count(*) AS CommentCount,
    Score
FROM 
    Comments
WHERE 
    UserId = @UserId
GROUP BY 
    Score
ORDER BY 
    Score DESC
END
GO




IF OBJECT_ID('dbo.usp_Q3160') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q3160 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q3160 @UserId INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/3160/jon-skeet-comparison */

with fights as (
  select myAnswer.ParentId as Question,
   myAnswer.Score as MyScore,
   jonsAnswer.Score as JonsScore
  from Posts as myAnswer
  inner join Posts as jonsAnswer
   on jonsAnswer.OwnerUserId = 22656 and myAnswer.ParentId = jonsAnswer.ParentId
  where myAnswer.OwnerUserId = @UserId and myAnswer.PostTypeId = 2
)

select
  case
   when MyScore > JonsScore then 'You win'
   when MyScore < JonsScore then 'Jon wins'
   else 'Tie'
  end as 'Winner',
  Question as [Post Link],
  MyScore as 'My score',
  JonsScore as 'Jon''s score'
from fights;
END
GO




IF OBJECT_ID('dbo.usp_Q6627') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q6627 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q6627 @Useless INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/6627/top-50-most-prolific-editors */

-- Top 50 Most Prolific Editors
-- Shows the top 50 post editors, where the user was the most recent editor
-- (meaning the results are conservative compared to the actual number of edits).

SELECT TOP 50
    Id AS [User Link],
    (
        SELECT COUNT(*) FROM Posts
        WHERE
            PostTypeId = 1 AND
            LastEditorUserId = Users.Id AND
            OwnerUserId != Users.Id
    ) AS QuestionEdits,
    (
        SELECT COUNT(*) FROM Posts
        WHERE
            PostTypeId = 2 AND
            LastEditorUserId = Users.Id AND
            OwnerUserId != Users.Id
    ) AS AnswerEdits,
    (
        SELECT COUNT(*) FROM Posts
        WHERE
            LastEditorUserId = Users.Id AND
            OwnerUserId != Users.Id
    ) AS TotalEdits
    FROM Users
    ORDER BY TotalEdits DESC

END
GO






IF OBJECT_ID('dbo.usp_Q6772') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q6772 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q6772 @UserId INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/6772/stackoverflow-rank-and-percentile */

WITH Rankings AS (
SELECT Id, Ranking = ROW_NUMBER() OVER(ORDER BY Reputation DESC)
FROM Users
)
,Counts AS (
SELECT Count = COUNT(*)
FROM Users
WHERE Reputation > 100
)
SELECT Id, Ranking, CAST(Ranking AS decimal(20, 5)) / (SELECT Count FROM Counts) AS Percentile
FROM Rankings
WHERE Id = @UserId

END
GO









IF OBJECT_ID('dbo.usp_Q6856') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q6856 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q6856 @MinReputation INT, @Upvotes INT = 100 AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/6856/high-standards-top-100-users-that-rarely-upvote */

select top 100
  Id as [User Link],
  round((100.0 * (Reputation/10)) / (UpVotes+1), 2) as [Ratio %],
  Reputation as Rep, 
  UpVotes as [+ Votes],
  DownVotes [- Votes]
from Users
where Reputation > @MinReputation
  and UpVotes > @Upvotes
order by [Ratio %] desc

END
GO




IF OBJECT_ID('dbo.usp_Q952') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q952 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q952 @Useless INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/952/top-500-answerers-on-the-site */

SELECT 
    TOP 500
    Users.Id as [User Link],
    Count(Posts.Id) AS Answers,
    CAST(AVG(CAST(Score AS float)) as numeric(6,2)) AS [Average Answer Score]
FROM
    Posts
  INNER JOIN
    Users ON Users.Id = OwnerUserId
WHERE 
    PostTypeId = 2 and CommunityOwnedDate is null and ClosedDate is null
GROUP BY
    Users.Id, DisplayName
HAVING
    Count(Posts.Id) > 10
ORDER BY
    [Average Answer Score] DESC

END
GO







IF OBJECT_ID('dbo.usp_Q975') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q975 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q975 @Useless INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/975/users-with-more-than-one-duplicate-account-and-a-more-than-1000-reputation-in-agg */

-- Users with more than one duplicate account and a more that 1000 reputation in aggregate
-- A list of users that have duplicate accounts on site, based on the EmailHash and lots of reputation is riding on it

SELECT 
    u1.EmailHash,
    Count(u1.Id) AS Accounts,
    (
        SELECT Cast(u2.Id AS varchar) + ' (' + u2.DisplayName + ' ' + Cast(u2.Reputation as varchar) + '), ' 
        FROM Users u2 
        WHERE u2.EmailHash = u1.EmailHash order by u2.Reputation desc FOR XML PATH ('')) AS IdsAndNames
FROM
    Users u1
WHERE
    u1.EmailHash IS NOT NULL
    and (select sum(u3.Reputation) from Users u3 where u3.EmailHash = u1.EmailHash) > 1000  
    and (select count(*) from Users u3 where u3.EmailHash = u1.EmailHash and Reputation > 10) > 1
GROUP BY
    u1.EmailHash
HAVING
    Count(u1.Id) > 1
ORDER BY 
    Accounts DESC

END
GO




IF OBJECT_ID('dbo.usp_Q8116') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_Q8116 AS RETURN 0;')
GO

ALTER PROC dbo.usp_Q8116 @UserId INT AS
BEGIN
/* Source: http://data.stackexchange.com/stackoverflow/query/8116/my-money-for-jam */

-- My Money for Jam
-- My Non Community Wiki Posts that earn the most Passive Reputation.
-- Reputation gained in the first 15 days of post is ignored,
-- all reputation after that is considered passive reputation.
-- Post must be at least 60 Days old.

set nocount on

declare @latestDate datetime
select @latestDate = max(CreationDate) from Posts
declare @ignoreDays numeric = 15
declare @minAgeDays numeric = @ignoreDays * 4

-- temp table moded from http://odata.stackexchange.com/stackoverflow/s/87
declare @VoteStats table (PostId int, up int, down int, CreationDate datetime)
insert @VoteStats
select
    p.Id,
    up = sum(case when VoteTypeId = 2 then
        case when p.ParentId is null then 5 else 10 end
        else 0 end),
    down = sum(case when VoteTypeId = 3 then 2 else 0 end),
    p.CreationDate
from Votes v join Posts p on v.PostId = p.Id
where v.VoteTypeId in (2,3)
and OwnerUserId = @UserId
and p.CommunityOwnedDate is null
and datediff(day, p.CreationDate, v.CreationDate) > @ignoreDays
and datediff(day, p.CreationDate, @latestDate) > @minAgeDays
group by p.Id, p.CreationDate, p.ParentId

set nocount off

select top 100 PostId as [Post Link],
  convert(decimal(10,2), up - down)/(datediff(day, vs.CreationDate, @latestDate) - @ignoreDays) as [Passive Rep Per Day],
  (up - down) as [Passive Rep],
  up as [Passive Up Reputation],
  down as [Passive Down Reputation],
  datediff(day, vs.CreationDate, @latestDate) - @ignoreDays as [Days Counted]
from @VoteStats vs
order by [Passive Rep Per Day] desc


END
GO


IF OBJECT_ID('dbo.usp_insertUsers') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_insertUsers AS RETURN 0;')
GO

ALTER PROC dbo.usp_insertUsers  AS
BEGIN

	/*
		Assign randomly 
		-- was sehe ich in Capture...wenn nur den Prozeduraufruf randomInsert oder randomQ dann muss ich drum herum arbeiten...z.B: Prozeduren auslagern in andere Datenbank
	*/
	
	/*
		Determine count of users added within the workload recorded
		This is a helper to get the workload and the inserts deterministic rather than having random inputs
	*/
	DECLARE @count INT = (SELECT COUNT(*) FROM dbo.Users WHERE CreationDate > '20200512') + 1;

	INSERT INTO	dbo.Users
	(
		Age,
		CreationDate,
		DisplayName,
		DownVotes,
		LastAccessDate,
		Reputation,
		UpVotes,
		[Views]
	)
	VALUES
	(
		37, -- Age - int
		'20200512 17:00', -- CreationDate - datetime
		'WORKLOADUSER' + CAST(@count AS VARCHAR(20)), -- DISPLAYNAME - NVARCHAR
		0, -- DownVotes - int
		'20200512 18:00', -- LastAccessDate - datetime
		@count*100, -- Reputation - int
		@count*2, -- UpVotes - int
		@count/20 -- Views - int
	)
	;
END


GO

IF OBJECT_ID('dbo.usp_insertPosts') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_insertPosts AS RETURN 0;')
GO

ALTER PROC dbo.usp_insertPosts  AS
BEGIN
	/*
		Determine count of users added within the workload recorded
		This is a helper to get the workload and the inserts deterministic rather than having random inputs
	*/
	DECLARE @count INT = (SELECT COUNT(*) FROM dbo.Posts WHERE CreationDate > '20200512') + 1;

	INSERT INTO dbo.Posts
	(
		Body,
		CreationDate,
		LastActivityDate,
		PostTypeId,
		Score,
		Tags,
		Title,
		ViewCount,
		OwnerUserId
	)
	VALUES
	(
		N'This is just a dummycontent', -- Body - nvarchar
		'20200512 18:00', -- CreationDate - datetime
		'20200512 18:00', -- LastActivityDate - datetime
		1, -- PostTypeId - int --stands for "Question"
		@count*100, -- Score - int
		N'sql-server-2019', -- Tags - nvarchar
		N'This is just a dummyquestion', -- Title - nvarchar
		@count*250, -- ViewCount - int
		212952 -- ok, well one user gets all the credit for the workload to be deterministic
		/*
		--randomly return one of the newly generated users
		(	SELECT TOP 1 id
			FROM dbo.Users u
			WHERE CreationDate > '20200501'
			ORDER BY NEWID()
		)
		*/
	);
END


GO

IF OBJECT_ID('dbo.usp_updatePosts') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_updatePosts AS RETURN 0;')
GO

ALTER PROC dbo.usp_updatePosts  AS
BEGIN
	UPDATE dbo.Posts
	SET
	    dbo.Posts.ViewCount = ViewCount +1
	WHERE 
		Id = 4315748;
END

IF OBJECT_ID('dbo.usp_insertVotes') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_insertVotes AS RETURN 0;')
GO

ALTER PROC dbo.usp_insertVotes  AS
BEGIN
	-- set up variables to randomly determine the votetype up (2) or down (3)
	DECLARE @voteTypeId INT = NULL; /* variable for vote type id*/


	/*
		Determine count of users added within the workload recorded
		This is a helper to get the workload and the inserts deterministic rather than having random inputs
	*/
	DECLARE @count INT = (SELECT COUNT(*) FROM dbo.Votes WHERE CreationDate > '20200512') + 1;

	-- let's be positive and achieve 2/3 positive votes (type 2) and 1/3 negative votes (type 3)
	IF (@count%3 = 0 OR @count%3=1) 
		SET @voteTypeId = 2;
	IF (@count%3 = 2)
		SET @voteTypeId = 3;

	-- debugging select

	INSERT INTO
	dbo.Votes
	(
		PostId,
		UserId,
		VoteTypeId,
		CreationDate
	)
	VALUES
	(	/*
   		-- randomly return one of the newly generated posts
		(	SELECT TOP 1 id
			FROM dbo.Posts 
			--WHERE CreationDate > '20200501'
			ORDER BY NEWID()
		)*/
		4315748, -- PostId - int
		61636 -- one users does all the votes for the workload to be deterministic
		/*
		-- randomly return one of the users
		(	SELECT TOP 1 id
			FROM dbo.Users 
			ORDER BY NEWID()
		)*/, -- UserId - int
		@voteTypeId, -- VoteTypeId - int 2 UpVote / 3 DownVote
		'20200512 18:15' -- CreationDate - datetime
	)
	;
END
GO

IF OBJECT_ID('dbo.usp_runStaticQueries') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_runStaticQueries AS RETURN 0;')
GO

ALTER PROCEDURE dbo.usp_runStaticQueries WITH RECOMPILE
AS
SET NOCOUNT ON
	-- this procedure runs all queries which are independent from a userid
	 EXEC dbo.usp_Q36660 @useless = 0;
	 EXEC dbo.usp_Q466 @useless = 0;
	 EXEC dbo.usp_Q6856 @minReputation = 1000;
	 EXEC dbo.usp_Q952 @useless = 0;
	 EXEC dbo.usp_Q975 @useless = 0;
GO



IF OBJECT_ID('dbo.usp_runUserQueries') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_runUserQueries AS RETURN 0;')
GO

ALTER PROCEDURE dbo.usp_runUserQueries WITH RECOMPILE
AS
SET NOCOUNT ON
	-- this procedure runs all queries which depend on a userid
	DECLARE @Id INTEGER = 212952; -- this is our colleague who creates all the posts in the workload
	EXEC dbo.usp_Q3160 @Id;
    EXEC dbo.usp_Q6772 @Id;
    EXEC dbo.usp_Q7521 @Id;
	EXEC dbo.usp_Q8116 @Id;
	EXEC dbo.usp_Q947 @Id;
	EXEC dbo.usp_Q949 @Id;
GO



/*
-- procedures for random workload (not suitable for workloadtesting and replay)

IF OBJECT_ID('dbo.usp_RandomInsert') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_RandomInsert AS RETURN 0;')
GO

ALTER PROCEDURE dbo.usp_RandomInsert WITH RECOMPILE
AS
SET NOCOUNT ON

DECLARE @Id INT = CAST(RAND() * 10000000 AS INT);
IF @Id % 3 = 0
    EXEC dbo.usp_insertVotes
ELSE IF @Id % 2 = 0
    EXEC dbo.usp_insertPosts
ELSE
	EXEC dbo.usp_insertUsers
GO

--EXEC dbo.usp_RandomInsert;



IF OBJECT_ID('dbo.usp_RandomQ') IS NULL
  EXEC ('CREATE PROCEDURE dbo.usp_RandomQ AS RETURN 0;')
GO

ALTER PROCEDURE dbo.usp_RandomQ WITH RECOMPILE
AS
SET NOCOUNT ON

DECLARE @Id INT =(	SELECT TOP 1 id			FROM dbo.Users 			ORDER BY NEWID()		);
IF @Id % 12 = 0
    EXEC dbo.usp_Q3160 @Id
ELSE IF @Id % 11 = 0
    EXEC dbo.usp_Q36660 @Id
ELSE IF @Id % 10 = 0
    EXEC dbo.usp_Q466 @Id
--ELSE IF @Id % 9 = 0
--    EXEC dbo.usp_Q6627 @Id
ELSE IF @Id % 8 = 0
    EXEC dbo.usp_Q6772 @Id
ELSE IF @Id % 7 = 0
    EXEC dbo.usp_Q6856 @Id
ELSE IF @Id % 6 = 0
    EXEC dbo.usp_Q7521 @Id
ELSE IF @Id % 5 = 0
    EXEC dbo.usp_Q8116 @Id
ELSE IF @Id % 4 = 0
    EXEC dbo.usp_Q947 @Id
ELSE IF @Id % 3 = 0
    EXEC dbo.usp_Q949 @Id
ELSE IF @Id % 2 = 0
    EXEC dbo.usp_Q952 @Id
ELSE
	EXEC dbo.usp_Q975 @Id
GO

--EXEC dbo.usp_RandomQ;



*/


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.151

Backup your database as basis for capture and replay

Note: This code is for SQL Server on Linux. If your SQL Server is running on Windows you have to adapt the directory (which is typical for democode anyway)

In [3]:
BACKUP DATABASE [StackOverflow2010] 
    TO DISK = N'/var/opt/mssql/backup/StackOverflow2010.bak' 
    WITH INIT, FORMAT, CHECKSUM, COMPRESSION, STATS=10;

10 percent processed.

20 percent processed.

30 percent processed.

40 percent processed.

50 percent processed.

...

100 percent processed.

Processed 1096528 pages for database 'StackOverflow2010', file 'StackOverflow2010' on file 1.

Processed 2 pages for database 'StackOverflow2010', file 'StackOverflow2010_log' on file 1.

BACKUP DATABASE successfully processed 1096530 pages in 45.475 seconds (188.381 MB/sec).

Total execution time: 00:00:45.695

Testing the Procedures

The Insert-Procedures create a record with dummy values in the tables Users, Posts and Votes.
The Update-Procedure increments the column ViewCount in the Posts table for one specific post.
The Select-Proecedures access all three tables regularly.


In [17]:
EXECUTE dbo.usp_insertUsers;
SELECT TOP 10 * FROM dbo.Users ORDER BY id DESC ;


EXECUTE dbo.usp_insertPosts;
SELECT TOP 10 OwnerUserId, * FROM dbo.Posts ORDER BY id DESC ;


EXECUTE dbo.usp_insertVotes;
SELECT TOP 10  * FROM dbo.Votes ORDER BY id DESC ;


(1 row affected)

(10 rows affected)

(1 row affected)

(10 rows affected)

(1 row affected)

(10 rows affected)

Total execution time: 00:00:03.869

Id AboutMe Age CreationDate DisplayName DownVotes EmailHash LastAccessDate Location Reputation UpVotes Views WebsiteUrl AccountId 10251167 NULL 37 2020-05-12 17:00:00.000 WORKLOADUSER3 0 NULL 2020-05-12 18:00:00.000 NULL 300 6 0 NULL NULL 10251166 NULL 37 2020-05-12 17:00:00.000 WORKLOADUSER2 0 NULL 2020-05-12 18:00:00.000 NULL 200 4 0 NULL NULL 10251165 NULL 37 2020-05-12 17:00:00.000 WORKLOADUSER1 0 NULL 2020-05-12 18:00:00.000 NULL 100 2 0 NULL NULL 10251164 NULL NULL 2010-11-09 22:17:33.317 Rune Jensen 0 NULL 2018-09-01 10:29:30.990 NULL 1 0 5 NULL 235581 10158561 NULL NULL 2010-01-29 10:10:42.217 user10158561 0 NULL 2018-08-10 13:13:24.423 NULL 1 0 6 http://www.google.com 96266 10029427 NULL NULL 2010-01-19 16:15:46.970 Travis Fischer 0 NULL 2018-08-14 23:33:36.477 Houston, TX 1 0 4 http://www.iovortex.com 92801 10000867 NULL 2010-06-24 10:23:41.537 Mukesh Singh 0 NULL 2018-08-05 15:54:52.950 Bangalore, Karnataka, India 1 0 7 157284 9827379 <p>iOS guy valiantly trying to wrestle JavaScript.</p>

<p>Also available in short format: <a href="http://twitter.com/#!/sockettrousers" rel="nofollow">sockettrousers</a> but that is really boring.</p>
 NULL 2009-02-05 15:28:06.897 Rog 446 NULL 2018-08-30 09:57:46.307 Ham Street, Kent 13240 1279 1619 http://www.ambientindustries.com 24680 9786914 Software Engineer, Mobile phone field. NULL 2009-12-29 02:42:01.063 Kevin 0 NULL 2018-05-14 08:01:39.293 Beijing, China 1 0 4 http://www.52bb.net.cn 86230 9782989 NULL 2010-09-29 20:56:51.460 GiladanoN 0 NULL 2018-08-30 10:04:30.333 NULL 1 0 3 210569

OwnerUserId Id AcceptedAnswerId AnswerCount Body ClosedDate CommentCount CommunityOwnedDate CreationDate FavoriteCount LastActivityDate LastEditDate LastEditorDisplayName LastEditorUserId OwnerUserId ParentId PostTypeId Score Tags Title ViewCount 212952 12496716 NULL NULL This is just a dummycontent NULL NULL NULL 2020-05-12 18:00:00.000 NULL 2020-05-12 18:00:00.000 NULL NULL NULL 212952 NULL 1 300 sql-server-2019 This is just a dummyquestion 750 212952 12496715 NULL NULL This is just a dummycontent NULL NULL NULL 2020-05-12 18:00:00.000 NULL 2020-05-12 18:00:00.000 NULL NULL NULL 212952 NULL 1 200 sql-server-2019 This is just a dummyquestion 500 212952 12496714 NULL NULL This is just a dummycontent NULL NULL NULL 2020-05-12 18:00:00.000 NULL 2020-05-12 18:00:00.000 NULL NULL NULL 212952 NULL 1 100 sql-server-2019 This is just a dummyquestion 250 163053 12496713 0 0 <p>I recently came upon <a href="http://www.cbs.dtu.dk/~eklund/beeswarm/">the beeswarm package</a>, that bears some similarity. </p>

<blockquote>
 <p>The bee swarm plot is a
 one-dimensional scatter plot like
 "stripchart", but with closely-packed,
 non-overlapping points.</p>
</blockquote>

<p>Here's an example:</p>

<pre><code> library(beeswarm)
 beeswarm(time_survival ~ event_survival, data = breast,
 method = 'smile',
 pch = 16, pwcol = as.numeric(ER),
 xlab = '', ylab = 'Follow-up time (months)',
 labels = c('Censored', 'Metastasis'))
 legend('topright', legend = levels(breast$ER),
 title = 'ER', pch = 16, col = 1:2)
</code></pre>

<p><a href="http://www.cbs.dtu.dk/~eklund/beeswarm/beeswarm_example_03.png">http://www.cbs.dtu.dk/~eklund/beeswarm/beeswarm_example_03.png</a></p>
 NULL 2 NULL 2010-10-14 14:55:28.180 0 2010-10-14 14:55:28.180 NULL NULL 0 163053 12496709 2 8 NULL NULL 0 370756 12496712 0 0 <p>I have come up with the code similar to Joris, still I think this is more than a stem plot; here I mean that they y value in each series is a absolute value of a distance to the in-bin mean, and x value is more about whether the value is lower or higher than mean.<br>
Example code (sometimes throws warnings but works):</p>

<pre><code>px&lt;-function(x,N=40,...){
x&lt;-sort(x);

#Cutting in bins
cut(x,N)-&gt;p;

#Calculate the means over bins
sapply(levels(p),function(i) mean(x[p==i]))-&gt;meansl;
means&lt;-meansl[p];

#Calculate the mins over bins
sapply(levels(p),function(i) min(x[p==i]))-&gt;minl;
mins&lt;-minl[p];

#Each dot is one value.
#X is an order of a value inside bin, moved so that the values lower than bin mean go below 0
X&lt;-rep(0,length(x));
for(e in levels(p)) X[p==e]&lt;-(1:sum(p==e))-1-sum((x-means)[p==e]&lt;0);
#Y is a bin minum + absolute value of a difference between value and its bin mean
plot(X,mins+abs(x-means),pch=19,cex=0.5,...);
}
</code></pre>
 NULL 2 NULL 2010-09-02 14:05:11.540 0 2010-09-02 14:05:11.540 NULL NULL 0 370756 12496709 2 4 NULL NULL 0 428790 12496711 0 0 <p>Make.Funny.Plot does more or less what I think it should do. To be adapted according to your own needs, and might be optimized a bit, but this should be a nice start.</p>

<pre><code>Make.Funny.Plot &lt;- function(x){
 unique.vals &lt;- length(unique(x))
 N &lt;- length(x)
 N.val &lt;- min(N/20,unique.vals)

 if(unique.vals&gt;N.val){
 x &lt;- ave(x,cut(x,N.val),FUN=min)
 x &lt;- signif(x,4)
 }
 # construct the outline of the plot
 outline &lt;- as.vector(table(x))
 outline &lt;- outline/max(outline)

 # determine some correction to make the V shape,
 # based on the range
 y.corr &lt;- diff(range(x))*0.05

 # Get the unique values
 yval &lt;- sort(unique(x))

 plot(c(-1,1),c(min(yval),max(yval)),
 type="n",xaxt="n",xlab="")

 for(i in 1:length(yval)){
 n &lt;- sum(x==yval[i])
 x.plot &lt;- seq(-outline[i],outline[i],length=n)
 y.plot &lt;- yval[i]+abs(x.plot)*y.corr
 points(x.plot,y.plot,pch=19,cex=0.5)
 }
}

N &lt;- 500
x &lt;- rpois(N,4)+abs(rnorm(N))
Make.Funny.Plot(x)
</code></pre>

<p>EDIT : corrected so it always works.</p>
 NULL 10 NULL 2010-09-02 12:46:29.

Id PostId UserId BountyAmount VoteTypeId CreationDate 33551690 4315748 61636 NULL 2 2020-05-12 18:15:00.000 33551689 13395409 61636 NULL 3 2020-05-12 18:15:00.000 33551688 13395409 61636 NULL 2 2020-05-12 18:15:00.000 33551687 12496713 NULL NULL 2 2010-10-14 00:00:00.000 33551686 12496713 NULL NULL 2 2010-10-14 00:00:00.000 33551685 12496713 NULL NULL 2 2010-10-14 00:00:00.000 33551684 12496713 NULL NULL 2 2010-10-14 00:00:00.000 33551683 12496713 NULL NULL 2 2010-10-14 00:00:00.000 33551682 12496713 NULL NULL 2 2010-10-14 00:00:00.000 33551680 12496712 NULL NULL 2 2010-09-03 00:00:00.000

In [19]:
USE [StackOverflow2010];
SELECT ViewCount FROM dbo.Posts WHERE id = 4315748;
EXECUTE dbo.usp_updatePosts;
SELECT ViewCount FROM dbo.Posts WHERE id = 4315748;



(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.009

ViewCount 1

ViewCount 2

Die Prozedur usp_runUserQueries führt alle Select-Abfragen aus, die User-bezogen sind. Hier wird der Einfachheit ein statischer User hinterlegt, damit sich der Workload zwischen Aufnahme und Wiederabspielen nicht verändert.

In [6]:
EXECUTE dbo.usp_runUserQueries;

(10 rows affected)

Total execution time: 00:01:01.212

Winner Post Link My score Jon's score Jon wins 3803135 2 9 Jon wins 3959206 5 11

Id Ranking Percentile 212952 7321 0,0541197865073850

Accepted Answers Scored Answers Unscored Answers Percentage Unscored 46 42 4 8,600000

Post Link Passive Rep Per Day Passive Rep Passive Up Reputation Passive Down Reputation Days Counted 4267473 0,0029052876234747239976 10 10 0 3442 4087694 0,0028876696505919722783 10 10 0 3463 4085352 0,0028876696505919722783 10 10 0 3463 4085193 0,0028876696505919722783 10 10 0 3463 4049723 0,0028835063437139561707 10 10 0 3468 3820561 0,0028587764436821040594 10 10 0 3498 3801740 0,0028571428571428571428 10 10 0 3500 3726634 0,0028481913984619766448 10 10 0 3511 3665575 0,0028417163967036089798 10 10 0 3519 3969920 0,0023001725129384703852 8 10 2 3478

CommentCount Score 1 9 1 5 1 3 3 2 10 1 108 0

AcceptedPercentage 42,9906542056075

Die Prozedur usp_runStaticQueries führt alle Select-Abfragen aus, die unabhängig von einem bestimmten Benutzer sind.

In [7]:
EXECUTE dbo.usp_runStaticQueries;

(100 rows affected)

Total execution time: 00:00:13.727

Vote count Post Link Body 70 3905734 <p>How can one send an email to 100,000 users on a weekly basis in PHP? This includes mail to subscribers using the following providers:</p>

<ul>
<li>AOL</li>
<li>G-Mail</li>
<li>Hotmail</li>
<li>Yahoo</li>
</ul>

<p>It is important that all e-mail actually be delivered, to the extent that it is possible. Obviously, just sending the mail conventionally would do nothing but create problems.</p>

<p>Is there a library for PHP that makes this simpler? </p>
 42 84556 <p>Personally I like this one:</p>

<p><a href="http://www.jeffpalm.com/fox/fox.jpg" rel="nofollow noreferrer">alt text http://www.jeffpalm.com/fox/fox.jpg</a></p>

<p>P.S. Do not hotlink the cartoon without the site's permission please. </p>
 28 343061 <p>I have the need/desire to learn to program against Win32 in C++. I am a little confused as to what Win32 even is, as I have no experience on the platform. </p>

<p>What would you recommend to get me started programming and debugging C++ programs on Win32?</p>
 28 1995113 <p>What is, in your opinion, the most surprising, weird, strange or really "WTF" language feature you have encountered?</p>

<p><strong>Please only one feature per answer.</strong></p>
 27 1003841 <p>How do I move the <a href="http://en.wikipedia.org/wiki/Turtle_graphics" rel="nofollow noreferrer">turtle</a> in <a href="http://en.wikipedia.org/wiki/Logo_(programming_language)" rel="nofollow noreferrer">LOGO</a>?</p>
 25 260511 <p>Here is my short implementation of <a href="http://en.wikipedia.org/wiki/Ancient_Egyptian_multiplication" rel="noreferrer">Russian Peasant Multiplication</a>. How can it be improved?</p>

<p><em>Restrictions</em> : only works when a>0,b>0</p>

<pre><code>for(p=0;p+=(a&amp;1)*b,a!=1;a&gt;&gt;=1,b&lt;&lt;=1);
</code></pre>
 23 406760 <p>This is definitely subjective, but I'd like to try to avoid it becoming argumentative. I think it could be an interesting question if people treat it appropriately.</p>

<p>The idea for this question came from the comment thread from <a href="https://stackoverflow.com/questions/282329/what-are-five-things-you-hate-about-your-favorite-language#282342">my answer</a> to the <a href="https://stackoverflow.com/questions/282329">"What are five things you hate about your favorite language?" question</a>. I contended that classes in C# should be sealed by default - I won't put my reasoning in the question, but I might write a fuller explanation as an answer to this question. I was surprised at the heat of the discussion in the comments (25 comments currently).</p>

<p>So, what contentious opinions do <em>you</em> hold? I'd rather avoid the kind of thing which ends up being pretty religious with relatively little basis (e.g. brace placing) but examples might include things like "unit testing isn't actually terribly helpful" or "public fields are okay really". The important thing (to me, anyway) is that you've got reasons behind your opinions.</p>

<p>Please present your opinion and reasoning - I would encourage people to vote for opinions which are well-argued and interesting, whether or not you happen to agree with them.</p>
 21 390693 <p>I've been fiddling with ASP.NET MVC since the CTP, and I like a lot of things they did, but there are things I just don't get.</p>

<p>For example, I downloaded beta1, and I'm putting together a little personal site/resume/blog with it. Here is a snippet from the ViewSinglePost view:</p>

<pre><code> &lt;%
 // Display the "Next and Previous" links
 if (ViewData.Model.PreviousPost != null || ViewData.Model.NextPost != null)
 {
 %&gt; &lt;div&gt; &lt;%

 if (ViewData.Model.PreviousPost != null)
 {
 %&gt; &lt;span style="float: left;"&gt; &lt;%
 Response.Write(Html.ActionLink("&lt;&lt; " + ViewData.Model.PreviousPost.Subject, "view", new { id = ViewData.Model.PreviousPost.Id }));
 %&gt; &lt;/span&gt; &lt;%
 }

 if (ViewData.Model.NextPost != null)
 {
 %&gt; &lt;span style="float: right;"&gt; &lt;%
 Response.Write(Html.ActionLink(Vie

Post Link up down 1135224 48 32 273522 38 22 260235 23 17 3130983 22 12 31082 22 17 260511 22 25 1783472 22 12 145177 21 16 3165311 21 13 193284 20 11 701340 20 20 269003 19 13 731771 17 10 1458904 17 11 4037003 17 10 3257237 17 10 1554822 17 9 1724280 16 10 71251 16 12 127176 15 17 694657 15 9 1827429 15 10 205373 15 10 3736094 15 8 1639957 15 8 377239 15 13 221848 15 8 115499 15 10 2494170 15 10 846015 14 12 1088250 14 8 858572 14 9 988623 14 9 56 14 9 96137 13 13 98881 13 10 1030388 13 7 3346859 13 7 159747 13 7 336275 13 7 1613247 13 7 3533838 13 7 234080 13 9 160439 13 9 1800013 13 8 3655 13 8 1161960 13 9 1400614 13 9 885918 13 12 1365320 12 10 587049 12 8 141151 12 12 2841697 12 10 282668 12 11 101071 12 8 326799 12 8 1082353 12 12 923039 12 13 956439 12 7 363250 12 7 119855 12 8 551303 12 11 1152230 11 6 182264 11 7 294210 11 7 2304941 11 6 1945667 11 8 159804 11 9 3793133 11 13 2894526 11 6 927378 11 6 463030 11 10 1348080 11 11 541184 11 6 53463 11 10 340299 11 12 289603 11 11 2877663 11 8 993124 11 9 579477 11 8 456876 11 9 231250 11 7 763895 11 8 376621 11 7 454574 11 6 152481 11 9 1365276 11 9 962129 11 8 245340 11 14 40736 11 6 3632726 10 7 1556264 10 9 2002647 10 7 3609438 10 6 230584 10 7 3961467 10 6 1229297 10 6 312881 10 10 831528 10 6 2988893 10 9

User Link Ratio % Rep + Votes - Votes 69083 10902,020000000000 538569 493 140 5445 10131,020000000000 568353 560 6 131527 9108,890000000000 204957 224 292 75170 7427,870000000000 226553 304 13 95810 7271,410000000000 597714 821 50 298575 5942,570000000000 87953 147 34 11465 5100,970000000000 52549 102 49 92315 4875,190000000000 129683 265 53 7412 4582,320000000000 181463 395 326 187690 4540,250000000000 251530 553 478 14113 4474,590000000000 54597 121 4 19563 4256,660000000000 489092 1148 203 29407 4172,640000000000 814505 1951 2651 4883 4155,700000000000 123841 297 36 308661 4064,710000000000 41467 101 43 350858 3988,460000000000 82964 207 9 338204 3689,320000000000 38006 102 3 17833 3645,050000000000 66342 181 26 148870 3543,300000000000 333073 939 266 182705 3536,440000000000 41731 117 32 4465 3412,180000000000 67225 196 30 58803 3406,980000000000 73254 214 114 53114 3344,580000000000 490655 1466 284 364001 3317,070000000000 40801 122 0 235349 3302,330000000000 42605 128 8 34549 3297,830000000000 45510 137 35 24181 3260,240000000000 246801 756 140 422133 3223,280000000000 37394 115 21 20862 3208,790000000000 558334 1739 73 176761 3203,470000000000 55422 172 24 111988 3192,040000000000 36076 112 9 22850 3189,860000000000 44020 137 40 1196 3181,730000000000 95779 300 50 133910 3140,360000000000 52133 165 12 164835 3064,680000000000 138835 452 115 453596 3014,710000000000 30757 101 9 236465 3011,180000000000 48488 160 22 1423 2995,240000000000 44036 146 20 27929 2943,610000000000 39159 132 56 122428 2930,910000000000 48364 164 195 149808 2869,270000000000 58824 204 22 128141 2847,880000000000 46991 164 72 263521 2847,660000000000 36458 127 2 38896 2763,570000000000 35654 128 7 1109 2763,480000000000 112751 407 136 35501 2759,010000000000 61254 221 2 368630 2757,950000000000 83298 301 70 149341 2723,600000000000 140811 516 3239 4482 2699,680000000000 85046 314 86 277683 2683,450000000000 37305 138 22 415448 2668,370000000000 104608 391 450 3560 2667,140000000000 150966 565 113 229410 2645,070000000000 37569 141 338 192 2621,570000000000 26747 101 9 2988 2606,850000000000 281545 1079 8273 331052 2595,360000000000 39197 150 18 18528 2594,070000000000 30617 117 26 383478 2577,610000000000 34543 133 6 54017 2557,080000000000 54212 211 7 12469 2505,770000000000 26060 103 31 259038 2437,940000000000 118242 484 9 55943 2387,160000000000 52040 217 648 398309 2385,340000000000 27670 115 1 1898 2383,330000000000 42904 179 8 136829 2382,660000000000 41228 172 849 11549 2360,000000000000 37764 159 35 82156 2348,330000000000 63173 268 13 121747 2342,570000000000 105656 450 89 511300 2337,780000000000 31561 134 22 11858 2305,960000000000 54192 234 60 249991 2299,300000000000 32652 141 27 153982 2296,310000000000 137090 596 55 348785 2268,750000000000 98016 431 13 10322 2251,920000000000 23425 103 5 20479 2229,270000000000 27427 122 10 467473 2221,050000000000 46427 208 27 147356 2178,960000000000 104597 479 2560 27204 2177,070000000000 57915 265 6 105999 2167,780000000000 39025 179 67 524555 2156,150000000000 52614 243 18 7136 2150,750000000000 28824 133 17 429 2112,790000000000 54510 257 157 32632 2104,640000000000 108810 516 210 17693 2089,470000000000 23828 113 1 19435 2065,690000000000 21071 101 101 598 2065,120000000000 44401 214 13 33518 2055,410000000000 193628 941 98 77567 2054,780000000000 281716 1370 228 44990 2054,290000000000 28765 139 4 18393 2051,340000000000 491096 2393 395 107366 2049,460000000000 37719 183 2 19268 2044,370000000000 29033 141 16 151312 2043,910000000000 47016 229 6 223092 2040,030000000000 271129 1328 3 9990 2033,430000000000 139900 687 121 32484 2031,360000000000 23974 117 10 406857 2021,850000000000 30531 150 18 493707 2006,670000000000 21075 104 12 217332 2006,230000000000 57987 288 61 489564 2005,040000000000 79607 396 16

User Link Answers Average Answer Score 95592 46 388,04 25449 24 351,38 214090 28 290,18 2030 14 237,79 289651 12 219,75 20479 12 188,67 231078 17 187,29 264 19 186,21 98068 18 182,78 39933 51 169,35 250096 11 165,91 462 21 165,10 8985 110 159,94 446 13 153,85 400653 12 153,67 22850 29 149,86 19082 34 141,35 36272 14 140,36 54187 11 136,91 12089 33 135,30 9897 12 131,08 3445 14 128,71 101290 11 127,18 1057 80 125,94 5422 35 125,37 822 53 119,74 32536 13 118,85 19235 19 118,79 20846 30 112,10 1879 25 111,92 25472 20 107,70 3785 20 105,55 177019 12 105,00 964 11 100,73 95706 105 99,43 101 29 98,93 58803 72 98,86 41165 26 98,54 137902 35 97,09 122683 24 94,46 7779 11 94,27 18093 11 92,82 92315 138 91,82 4642 63 91,78 17801 25 91,64 9951 261 90,19 13892 12 89,83 28324 27 89,07 172594 14 88,00 402 14 87,86 32899 17 87,65 185196 12 87,08 40764 11 86,82 128988 18 86,39 5536 48 86,10 183801 12 85,17 14755 23 82,96 235945 16 82,88 9084 17 81,76 14193 11 80,27 12123 14 80,14 21590 27 79,41 13760 22 78,73 150650 11 78,64 363 21 78,33 44563 18 78,22 15690 18 77,44 110223 11 76,55 174576 14 75,86 31563 20 75,55 15754 12 73,92 346573 42 73,14 1736623 11 72,73 2342 24 72,67 22710 12 72,08 19104 20 71,65 148314 12 71,50 106673 14 71,21 7748 18 71,00 13673 77 70,91 825 138 70,57 116838 25 69,64 429 73 69,62 17833 53 69,15 204414 22 69,05 256130 12 68,58 1199 200 68,38 56018 66 67,92 49590 15 67,80 4120 72 67,40 69339 14 67,29 31625 12 67,00 250683 13 66,08 11830 94 66,07 151382 20 64,45 5752 11 64,18 11858 79 63,14 147458 11 62,73 108802 23 62,13 108648 22 61,50 23963 17 61,35 369986 19 60,74 2338 35 60,46 65454 11 60,45 1987 18 60,11 373 24 59,96 5026 13 59,77 2900 40 59,75 9410 119 59,71 30497 16 59,56 102939 20 59,15 41191 11 59,09 22368 13 59,08 28840 14 58,93 184367 16 58,88 27580 74 57,57 135965 31 57,55 37664 13 57,31 158773 44 57,11 18515 87 57,07 86463 120 56,84 18039 13 56,77 242583 18 56,72 1310 86 56,66 2628 22 56,59 3093 56 56,39 358281 37 56,32 8763 11 56,00 95380 16 55,81 109869 34 55,50 216 11 55,45 309853 19 55,16 1775 23 55,09 205768 12 54,92 7829 11 54,82 20089 35 54,74 6682 34 54,32 164802 15 54,07 61008 30 54,03 5674 15 53,47 27565 18 53,33 268330 37 53,30 119901 11 53,18 183994 18 53,00 1941213 74 52,65 19468 37 52,57 4172 94 52,51 40618 20 52,40 50450 11 52,36 24630 16 52,31 5821 45 52,29 1466 34 52,18 119963 501 52,16 46058 334 52,04 22244 12 51,92 111896 58 51,74 155351 50 51,64 720 55 51,60 69689 22 50,86 22445 20 50,80 190371 17 49,82 19130 21 49,76 6083 15 49,67 130961 25 49,64 279452 21 49,43 118962 16 49,06 85509 41 49,00 9414 28 48,93 171675 16 48,88 51051 15 48,80 69768 30 48,73 218783 14 48,29 37522 13 48,15 12682 44 48,07 105562 18 47,94 4223 66 47,52 85360 70 47,34 4883 19 47,26 19239 13 47,08 17815 61 47,08 4766 27 46,74 135724 44 46,61 93802 13 46,54 71116 25 46,52 2089740 17 46,24 287279 25 46,20 129190 69 45,94 21279 11 45,73 4048 15 45,60 20984 105 45,11 134360 35 45,11 142 23 44,96 210303 27 44,85 105976 23 44,83 384797 23 44,65 79595 28 44,64 116 241 44,61 33036 71 44,46 98654 43 44,37 12469 51 44,22 40005 187 44,02 22227 12 43,92 3045 12 43,83 381619 19 43,74 64417 19 43,74 2806 12 43,58 251956 20 43,40 285694 20 43,40 207847 18 43,28 24789 18 43,11 7496 11 43,09 24179 22 43,09 271974 15 42,80 1423 100 42,69 44597 20 42,65 24820 19 42,58 48492 165 42,51 207432 85 42,35 36433 62 42,34 20789 95 42,32 103421 19 42,26 1694 232 42,20 35189 16 42,13 218340 22 42,09 11138 45 41,91 56829 42 41,90 63147 79 41,84 118 22 41,68 39365 25 41,64 20074 87 41,53 136 29 41,45 5382 28 41,43 7980 20 41,40 2010 63 41,35 19563 1073 41,18 51103 42 41,14 104184 187 41,06 86295 13 40,92 16686 11 40,91 162796 26 40,65 216431 22 40,55 353004 14 40,50 4999 47 40,43 5704 43 40,33 24806 18 40,28 42891 21 40,19 233293 20 40,00 48728 38 39,89 8930 47 39,87 514553 47 39,83 63557 51 39,78 20377 42 39,62 28158 29 39,62 54937 35 39,54 299509 131 39,46 8922 11 39,45 3 28 39,43 184046 33 39,30 122169 21 39,29 4465 142 39,25 58347 33 39,24 

EmailHash Accounts IdsAndNames